In [2]:
# This function checks if two points are within a certain range of one another.
def in_range(point1, point2, tolerance):
    y_scale = 3

    x_within_range = point1[0] - tolerance <= point2[0] <= point1[0] + tolerance
    y_within_range = point1[1] - tolerance * y_scale <= point2[1] <= point1[1] + tolerance * y_scale

    return x_within_range and y_within_range


# This function calculates Intersection over Union (IoU) between two bounding boxes.
def compute_iou(box_base, box_unknown):
    x1, y1 = box_base[0] - box_base[2] / 2, box_base[1] - box_base[3] / 2
    x2, y2 = box_base[0] + box_base[2] / 2, box_base[1] + box_base[3] / 2
    x3, y3 = box_unknown[0] - box_unknown[2] / 2, box_unknown[1] - box_unknown[3] / 2
    x4, y4 = box_unknown[0] + box_unknown[2] / 2, box_unknown[1] + box_unknown[3] / 2

    # Calculate area of intersection
    x_overlap = max(0, min(x2, x4) - max(x1, x3))
    y_overlap = max(0, min(y2, y4) - max(y1, y3))
    intersection_area = x_overlap * y_overlap

    # Calculate area of union
    box1_area = box_base[2] * box_base[3]
    box2_area = box_unknown[2] * box_unknown[3]
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    return iou


# This function scales the input image by the specified scale factor.
def scale_image(input_image, scale_factor):
    width = input_image.shape[1]
    height = input_image.shape[0]
    output_image = cv2.resize(input_image, (int(width * scale_factor), int(height * scale_factor)))

    return output_image


# Squares images based on length does not interact with one dimension
def square_images(input_images, resize_option=False):
    output_images = []

    for label in input_images:
        img_shape = input_images[label].shape

        # If the image is longer in width, reduce both sides equally to achieve desired size.
        if img_shape[0] < img_shape[1]:  # Longer in width
            reduction = int((img_shape[1] - img_shape[0]) / 2)

            input_images[label] = input_images[label][:, reduction: img_shape[0] + reduction]

        # If the image is taller in height, reduce both sides equally to achieve desired size.
        if img_shape[0] > img_shape[1]:  # Taller in height
            reduction = int((img_shape[0] - img_shape[1]) / 2)

            input_images[label] = input_images[label][reduction: img_shape[1] + reduction]

        # Resize all images to the same size if required
        if type(resize_option) == int:
            input_images[label] = cv2.resize(input_images[label], (resize_option, resize_option))

    return input_images

    
# Scales normalized data to the image
def scale_detections(detections, image):
    
    # Rescale detection coordinates based on the input image dimensions
    detections[1::2] = [int(x * image.shape[1]) for x in detections[1::2]]  # X-Values
    detections[2::2] = [int(y * image.shape[0]) for y in detections[2::2]]  # Y-Values

    return detections

# This function determines the most significant object in a set of images.
def most_significant_object(detections):
    machine_zero = 0.01

    unique_classes = []

    for i, detection in enumerate(detections):

        if detection is None or detection.shape[0] == 0:
            success = False
            return success, detections

        detection = np.array(detection)

        # Remove objects touching the image border
        border_mask = (detection[:, 1] - (detection[:, 3] / 2) >= machine_zero) \
                      & (1 - detection[:, 2] - (detection[:, 4] / 2) >= machine_zero) \
                      & (1 - detection[:, 1] - (detection[:, 3] / 2) >= machine_zero) \
                      & (detection[:, 2] - (detection[:, 4] / 2) >= machine_zero)

        # Apply mask to remove objects
        detections[i] = detection[border_mask]

        # Find unique classes in the image
        image_unique_classes = np.unique(np.array(detections[i][:, 0]))
        unique_classes.extend([*image_unique_classes])

    unique_classes = np.unique(unique_classes, return_counts=True)

    # Find the first class present in all images
    class_index = 0

    if len(unique_classes[0]) == 0:
        success = False
        return success, detections

    for i in range(len(unique_classes[0])):
        class_index = i

        if unique_classes[1][i] == 3:
            break
        if i == len(unique_classes[0]) - 1:
            success = False
            return success, detections

    common_class = unique_classes[0][class_index]

    output = []

    # Create output array containing the largest object of each image, assumed to be the same object
    for i, detection in enumerate(detections):
        class_mask = (detection[:, 0] == common_class)
        detection = detection[class_mask]

        # Sort objects by size
        detections[i] = np.array(sorted(detection, key=lambda det: det[-1] * det[-2], reverse=True))

        # Select the largest object from each image
        output.append(detections[i][0])

    tolerance = 0.15

    # Check if all objects are within range
    iou_values = [compute_iou(output[0][1:], output[1][1:]),
                  compute_iou(output[0][1:], output[2][1:]),
                  compute_iou(output[1][1:], output[2][1:])]

    min_iou = np.mean(iou_values)
    tolerance = 0.2

    success = in_range(output[0][1:3], output[1][1:3], tolerance) \
              & in_range(output[0][1:3], output[2][1:3], tolerance) \
              & in_range(output[1][1:3], output[2][1:3], tolerance)

    if min_iou > tolerance and success:
        success = True
    else:
        success = False

    output = np.array(output)

    # Return success flag and the output objects
    return success, output


def calibrate_images(images, model, minimal_img, depth_bias=1.0, view_relationship=True):
    images_used = ["RGB", "Thermal", "Left"]

    # Square incoming images to preserve ratio between all images
    images = square_images(images)

    multi_image_detections = []

    # Gather detections for each image
    for label in images_used:
        detection = model.predict(source=images[label], classes=[0, 2], verbose=False, device=0)
        detection = torch.cat((detection[0].boxes.cls.view(-1, 1), detection[0].boxes.xywhn), dim=1)
        multi_image_detections.append(detection.cpu())

    # Check if objects in images are identical
    success, multi_image_detections = most_significant_object(multi_image_detections)

    # If objects are not identical, return None
    if not success:
        return None, None

    multi_image_detections = {"RGB": multi_image_detections[0], "Thermal": multi_image_detections[1], "Left": multi_image_detections[2]}

    # Select the most important feature of the object, length or height
    important_feature = {2.0: -2, 0.0: -1}
    important_feature = important_feature[multi_image_detections[minimal_img][0]]

    # Find the smallest object, assume this camera holds the most information
    min_size = multi_image_detections[minimal_img][important_feature]
    min_idx = minimal_img

    # Scale detections of minimal image since it will not change
    multi_image_detections[minimal_img] = scale_detections(multi_image_detections[minimal_img], images[minimal_img])

    scales = []
    offsets = []

    # Calculate the relationship of each image in relation to the smallest
    for label in images_used:

        if label == minimal_img:
            # If the minimal image is encountered, set relationship to None
            scales.append(1)
            offsets.append([0, 0, 0, 0])
            continue

        # Calculate scale based on difference of feature size
        scale = (min_size * images[minimal_img].shape[0]) / (multi_image_detections[label][important_feature] * images[label].shape[0])

        if label == "Left":
            scale *= depth_bias

        # Resize images based on scale
        images[label] = scale_image(images[label].copy(), scale)

        # Scale detections after resizing
        single_image_detection = scale_detections(multi_image_detections[label], images[label])

        # Calculate difference in resolution
        resolution_diff_x = images[minimal_img].shape[1] - images[label].shape[1]
        resolution_diff_y = images[minimal_img].shape[0] - images[label].shape[0]

        # Compare offset of the object compared to minimal image
        offset_x = multi_image_detections[minimal_img][1] - single_image_detection[1]
        offset_y = multi_image_detections[minimal_img][2] - single_image_detection[2]

        # Calculate top, bottom, left, and right offsets
        top = int(offset_y)
        bottom = int(resolution_diff_y - offset_y)
        left = int(offset_x)
        right = int(resolution_diff_x - offset_x)

        # Create offset and scale values, this is the relationship between the images
        offsets.append([top, bottom, left, right])
        
            
        scales.append(scale)

        # Visualize the relationship if requested
        if view_relationship:
            relationship_img = images[label].copy()
            x = int(single_image_detection[1])
            y = int(single_image_detection[2])
            
            relationship_img = cv2.line(relationship_img, (x, y), (x + left, y), (0, 255, 0), 2)
            relationship_img = cv2.line(relationship_img, (x, y), (x, y + top), (0, 0, 255), 2)

            cv2.imshow("cali_" + label, relationship_img)
            cv2.waitKey(1)

    return scales, offsets


# Adjusts images based on the relationship information to match the minimal image
def modify_images(images, scales, offsets, min_image_size):
    images_used = ["RGB", "Thermal", "Depth"]

    # Modify images to match the minimal image
    for label in images_used:

        if scales[label] == 1:
            images[label] = square_images({label: images[label]})[label]
            continue

        # Scale images
        images[label] = scale_image(images[label], scales[label])

        top, bottom, left, right = offsets[label]

        # Calculate padding for the square image
        padding = int((images[label].shape[1] - images[label].shape[0]) / 2)

        # Calculate left padding taking into account the offset
        left_padding = padding - left

        # Crop the image horizontally
        images[label] = images[label][:, left_padding: left_padding + min_image_size]

        # Adjust right padding if necessary
        right = int(min_image_size - images[label].shape[1])

        # Adjust top and bottom padding
        if top < 0:
            images[label] = images[label][-top:min_image_size - top]
            top = 0
        else:
            images[label] = images[label][:min_image_size - top]

        bottom = int(min_image_size - images[label].shape[0] - top)

        # Add borders to image to fit, using abs for error prevention
        images[label] = cv2.copyMakeBorder(images[label], abs(top), abs(bottom), 0, abs(right), cv2.BORDER_CONSTANT, value=(0, 0, 0))

    return images


# Applies masks to the images to create the SII
def apply_mask(images, masks, main_img):

    # Copy the main image to avoid modifying the original
    super_imposed_image = images[main_img].copy()

    master_mask = []

    # Combine all masks
    if masks is not None:
        for m in masks:

            m = np.array(m)

            # Scale mask to match image size
            m = cv2.resize(m, super_imposed_image.shape[:2][::-1])

            if len(master_mask) == 0:
                master_mask = m
            else:
                master_mask += m

    else:
        return super_imposed_image

    # Invert master_mask to remove objects from the image
    master_mask = 1 - master_mask

    # Convert to bool to apply to images
    master_mask = master_mask.astype(bool)

    # Create masked copies of the thermal and depth images
    thermal_masked = images["Thermal"].copy()
    stereo_masked = images["Color_Depth"].copy()

    # Resize images if necessary
    if thermal_masked.shape != master_mask.shape:
        thermal_masked = cv2.resize(thermal_masked, master_mask.shape[:2][::-1])
        stereo_masked = cv2.resize(stereo_masked, master_mask.shape[:2][::-1])
        super_imposed_image = cv2.resize(super_imposed_image, master_mask.shape[:3][::-1])

    # Set pixels not related to objects to zero
    thermal_masked[master_mask] = 0
    stereo_masked[master_mask] = 0

    # Display the images to check if the mask is working correctly
    cv2.imshow("thermalMasked", thermal_masked)
    cv2.imshow("stereoMasked", stereo_masked)
    cv2.waitKey(1)
    
    # Apply the depth mask to the super_imposed image
    mask = stereo_masked.astype(bool)
    super_imposed_image[mask] = cv2.addWeighted(super_imposed_image, 0.70, stereo_masked, 0.30, 0)[mask]
    
    # Apply the thermal mask to the super_imposed image
    mask = thermal_masked.astype(bool)
    super_imposed_image[mask] = cv2.addWeighted(super_imposed_image, 0.65, thermal_masked, 0.35, 0)[mask]

    # Return the final super_imposed image
    return super_imposed_image
    
    
def superimpose_images(images, resize=512, add_color=False):
    # Resize images to the specified size and make them square
    images = square_images(images, resize)

    if add_color:
        # Normalize the depth image, enhance contrast and apply color map
        images["Depth"] = cv2.normalize(images["Depth"], None, 255, 0, cv2.NORM_INF, cv2.CV_8UC1)
        images["Depth"] = cv2.equalizeHist(images["Depth"])
        images["Depth"] = cv2.applyColorMap(images["Depth"], cv2.COLORMAP_JET)

    # Blend the RGB and depth images using weights, then blend the result with the thermal image
    superimposed_image = cv2.addWeighted(images["RGB"], 0.70, images["Depth"], 0.30, 0)
    superimposed_image = cv2.addWeighted(superimposed_image, 0.65, images["Thermal"], 0.35, 0)

    return superimposed_image

def filter_offsets_scales(TenScales, TenOffsets, scale_filter=np.median, offset_filter=np.median):
    # Filter scales by applying the scale_filter function to each component of the TenScales list
    filtered_scales = [scale_filter([i[0] for i in TenScales]),
                       scale_filter([i[1] for i in TenScales]),
                       scale_filter([i[2] for i in TenScales])]
    
    # Filter offsets by applying the offset_filter function to each component of the TenOffsets list
    filtered_offsets = []
    for i in range(3):
        # Extract the i-th component of each offset in TenOffsets and apply offset_filter to get the filtered value
        row = [offset_filter([j[i][k] for j in TenOffsets]) for k in range(4)]
        # Convert the float values to integers
        row = [int(x) for x in row]
        # Add the filtered row to the filtered_offsets list
        filtered_offsets.append(row)

    # Return the filtered scales and offsets
    return filtered_scales, filtered_offsets